<a href="https://colab.research.google.com/github/barronh/finn2cmaq/blob/master/examples/FINN2CMAQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example Application of FINN2CMAQ

    author: Barron H. Henderson
    date: 2021-06-24

## Objective

Prepare CMAQ-ready emissions from the Fire INventory from National Center for Atmospheric Research (FINN) version 1.5.

## Steps

1. Install necessary prerequisites.
2. Download FINN2CMAQ working directory.
3. Enter the working directory
4. Download 1-year of FINN data
5. Grid the data to a CMAQ domain.
6. Apply hourly allocation, vertical allocation and speciation.
7. Repeat the process for Near-Real-Time FINN

## Step 1: Install necessary prerequisites

In [1]:
%%capture
pip install pyproj pseudonetcdf pandas

## Step 2: Downlaod FINN2CMAQ working directory

In [2]:
import requests
import zipfile
import gc
from datetime import datetime

In [3]:
zresp = requests.get('https://codeload.github.com/barronh/finn2cmaq/zip/refs/heads/master')

In [4]:
with open('finn2cmaq-master.zip', 'wb') as zf:
  zf.write(zresp.content)

In [5]:
zipfile.ZipFile('finn2cmaq-master.zip').extractall()

## Step 3: Enter the working directory

In [6]:
cd finn2cmaq-master/

/content/finn2cmaq-master


## Step 4: Download 1-year of FINN data

Download FINN v1.5 with GEOS-Chem Speciation

1. Go to https://www.acom.ucar.edu/Data/fire/
2. Enter your contact info
3. Select "FINNv1.5 with Year: 2016 Speciation: GEOS-Chem
4. Enter the security question answer
5. Click submit
6. Right click on the "Click Here" do download and copy the URL
7. Paste the url below

In [7]:
finnurl = 'https://www.acom.ucar.edu/Data/fire/data/finn1/FINNv1.5_2016.GEOSCHEM.tar.gz'

In [8]:
!wget -r --continue {finnurl}

--2021-06-24 15:00:50--  https://www.acom.ucar.edu/Data/fire/data/finn1/FINNv1.5_2016.GEOSCHEM.tar.gz
Resolving www.acom.ucar.edu (www.acom.ucar.edu)... 128.117.164.26
Connecting to www.acom.ucar.edu (www.acom.ucar.edu)|128.117.164.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 988875973 (943M) [application/x-gzip]
Saving to: ‘www.acom.ucar.edu/Data/fire/data/finn1/FINNv1.5_2016.GEOSCHEM.tar.gz’

www.acom.ucar.edu/D 100%[===================>] 943.07M  77.8MB/s    in 12s     

2021-06-24 15:01:03 (75.9 MB/s) - ‘www.acom.ucar.edu/Data/fire/data/finn1/FINNv1.5_2016.GEOSCHEM.tar.gz’ saved [988875973/988875973]

FINISHED --2021-06-24 15:01:03--
Total wall clock time: 13s
Downloaded: 1 files, 943M in 12s (75.9 MB/s)


## Step 5: Grid the data to a CMAQ domain.

* Print run help.
* Grid 2016 data (takes ~2min)
  * To use a different grid, edit GRIDDESC

As a note, I precede many `run` calls with a `gc.collect()`. This ensures that unused memory is cleared before starting a command. This matters most if you are *rerunning* cells.

In [9]:
%run scripts/txt2daily.py -h

usage: txt2daily.py [-h] [-v] GRIDDESC GDNAM YEAR FINNPATH OUTPATH

positional arguments:
  GRIDDESC
  GDNAM
  YEAR
  FINNPATH
  OUTPATH

optional arguments:
  -h, --help     show this help message and exit
  -v, --verbose


In [10]:
gc.collect()
%run -t scripts/txt2daily.py aux/GRIDDESC 108NHEMI2 2016 www.acom.ucar.edu/Data/fire/data/finn1/FINNv1.5_2016.GEOSCHEM.tar.gz ./gridded/FINNv1.5_2016.GEOSCHEM.nc

**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  IOAPI_ISPH is assumed to be 6370000.; consistent with WRF
**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  IOAPI_ISPH is assumed to be 6370000.; consistent with WRF
**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  Currently not using:straight_vertical_longitude_from_pole -98.0



IPython CPU timings (estimated):
  User   :     106.60 s.
  System :       6.22 s.
Wall time:     112.40 s.


## Step 6: Apply hourly allocation, vertical allocation and speciation.

* Print run help.
* Run for 2016 daily (takes ~30s per day, so 3h for a year).
  * For example purposes, I run the first day of Jan, Apr, Jun, and Oct.
  * If you omit `-d`, it will run all days.

In [12]:
%run scripts/daily2hourly3d.py -h

usage: daily2hourly3d.py [-h] [-v] [-l LAYERPATH] [-t TPROPATH] [-e EXPRPATH]
                         [-d DATES]
                         inpath outtmp

positional arguments:
  inpath
  outtmp

optional arguments:
  -h, --help            show this help message and exit
  -v, --verbose
  -l LAYERPATH, --layerpath LAYERPATH
                        Layer fraction file
  -t TPROPATH, --tpropath TPROPATH
                        Hourly temporal profile path
  -e EXPRPATH, --exprpath EXPRPATH
                        Speciation conversion script path
  -d DATES, --date DATES
                        Process only this date YYYY-MM-DD


In [17]:
gc.collect()
%run -t scripts/daily2hourly3d.py -d 2016-01-01 -d 2016-04-01 -d 2016-07-01 -d 2016-10-01 ./gridded/FINNv1.5_2016.GEOSCHEM.nc ./cmaqready/FINNv1.5_%Y-%m-%d.GEOSCHEM.nc

Preparing inputs and factors


**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  IOAPI_ISPH is assumed to be 6370000.; consistent with WRF
**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  Currently not using:straight_vertical_longitude_from_pole -98.0


Preparing outputs
Processing days
Processing 2016-01-01.AREA.CO.NO.NO2.SO2.NH3.ECH4.ACET.ALD2.BENZENE.PAR.ETHY.ETHE.ETHA.PRPA.FORM.GLY.GLYC.HAC.MGLY.KET.IOLE.TOL.XYL.POC.PEC.PSO4.PNO3.PMOTHR.Saving
Processing 2016-04-01.AREA.CO.NO.NO2.SO2.NH3.ECH4.ACET.ALD2.BENZENE.PAR.ETHY.ETHE.ETHA.PRPA.FORM.GLY.GLYC.HAC.MGLY.KET.IOLE.TOL.XYL.POC.PEC.PSO4.PNO3.PMOTHR.Saving
Processing 2016-07-01.AREA.CO.NO.NO2.SO2.NH3.ECH4.ACET.ALD2.BENZENE.PAR.ETHY.ETHE.ETHA.PRPA.FORM.GLY.GLYC.HAC.MGLY.KET.IOLE.TOL.XYL.POC.PEC.PSO4.PNO3.PMOTHR.Saving
Processing 2016-10-01.AREA.CO.NO.NO2.SO2.NH3.ECH4.ACET.ALD2.BENZENE.PAR.ETHY.ETHE.ETHA.PRPA.FORM.GLY.GLYC.HAC.MGLY.KET.IOLE.TOL.XYL.POC.PEC.PSO4.PNO3.PMOTHR.Saving

IPython CPU timings (estimated):
  User   :      74.03 s.
  System :       5.75 s.
Wall time:      79.78 s.


## Step 7: Repeat the process for Near-Real-Time

In [14]:
YYYYJJJ = datetime(2020, 3, 1).strftime('%Y%j')
!wget -r --continue https://www.acom.ucar.edu/acresp/MODELING/finn_emis_txt/GLOB_GEOSchem_{YYYYJJJ}.txt.gz

--2021-06-24 15:05:08--  https://www.acom.ucar.edu/acresp/MODELING/finn_emis_txt/GLOB_GEOSchem_2020061.txt.gz
Resolving www.acom.ucar.edu (www.acom.ucar.edu)... 128.117.164.26
Connecting to www.acom.ucar.edu (www.acom.ucar.edu)|128.117.164.26|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1691831 (1.6M) [application/x-gzip]
Saving to: ‘www.acom.ucar.edu/acresp/MODELING/finn_emis_txt/GLOB_GEOSchem_2020061.txt.gz’

www.acom.ucar.edu/a 100%[===================>]   1.61M  6.49MB/s    in 0.2s    

2021-06-24 15:05:09 (6.49 MB/s) - ‘www.acom.ucar.edu/acresp/MODELING/finn_emis_txt/GLOB_GEOSchem_2020061.txt.gz’ saved [1691831/1691831]

FINISHED --2021-06-24 15:05:09--
Total wall clock time: 0.5s
Downloaded: 1 files, 1.6M in 0.2s (6.49 MB/s)


In [15]:
gc.collect()
%run -t scripts/txt2daily.py aux/GRIDDESC 108NHEMI2 2020 www.acom.ucar.edu/acresp/MODELING/finn_emis_txt/GLOB_GEOSchem_2020061.txt.gz ./gridded/GLOB_GEOSchem_2020061.nc

**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  IOAPI_ISPH is assumed to be 6370000.; consistent with WRF
**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  IOAPI_ISPH is assumed to be 6370000.; consistent with WRF
**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  Currently not using:straight_vertical_longitude_from_pole -98.0



IPython CPU timings (estimated):
  User   :       0.52 s.
  System :       0.03 s.
Wall time:       0.56 s.


In [18]:
gc.collect()
%run -t scripts/daily2hourly3d.py ./gridded/GLOB_GEOSchem_2020061.nc ./cmaqready/GLOB_GEOSchem_%Y-%m-%d.nc

Preparing inputs and factors


**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  IOAPI_ISPH is assumed to be 6370000.; consistent with WRF
**PNC:/usr/local/lib/python3.7/dist-packages/PseudoNetCDF/pncwarn.py:24:UserWarning:
  Currently not using:straight_vertical_longitude_from_pole -98.0


Preparing outputs
Processing days
Processing 2020-03-01.AREA.CO.NO.NO2.SO2.NH3.ECH4.ACET.ALD2.BENZENE.PAR.ETHY.ETHE.ETHA.PRPA.FORM.GLY.GLYC.HAC.MGLY.KET.IOLE.TOL.XYL.POC.PEC.PSO4.PNO3.PMOTHR.Saving

IPython CPU timings (estimated):
  User   :      18.23 s.
  System :       0.92 s.
Wall time:      19.21 s.
